In [155]:
#Importing pandas library
# %pip install pandas
# %pip install openpyxl
import pandas as pd

#customer_for_teaching.csv link https://drive.google.com/file/d/1sTmR2hDHh88Fin1OXESlTT8mwYLQwaSZ/view?usp=drive_link
# !gdown --id 1sTmR2hDHh88Fin1OXESlTT8mwYLQwaSZ

In [156]:
# ETLDataSource1.xlsx
orderSource1 = pd.read_excel('Data/ETLDataSource1.xlsx', sheet_name='orderSource1', engine='openpyxl')
productSource1 = pd.read_excel('Data/ETLDataSource1.xlsx', sheet_name='productSource1', engine='openpyxl')
stateLookup = pd.read_excel('Data/ETLDataSource1.xlsx', sheet_name='StateLookup', engine='openpyxl')

In [157]:
# ETLDataSource2.xlsx
orderSource2 = pd.read_excel('Data/ETLDataSource2.xlsx', sheet_name='orderSource2', engine='openpyxl')
productSource2 = pd.read_excel('Data/ETLDataSource2.xlsx', sheet_name='productSource2', engine='openpyxl')

In [158]:
orderSource1.head()

,OrderID,CustomerName,CustomerCity,CustomerState,CustomerStatus,OrderDate
0,10248,Suzan Plock,Pittsburgh,PA,Silver,2023-01-11
1,10249,Allan Strate,Miami,FL,Platinum,2023-01-11
2,10250,Elnora Willison,Philadelphia,PA,Platinum,2023-01-11
3,10251,Daniela Becknell,Tacoma,WA,Platinum,2023-01-11
4,10252,Cathrine Delamater,Philadelphia,PA,Platinum,2023-01-11


In [159]:
orderSource2.head()

,OrderID,CustomerFirstName,CustomerLastName,CustomerCity,CustomerState,CustomerStatus,OrderDate
0,A10258,Milagros,Fehrenbach,Kansas City,Kansas,2,2023-01-11
1,A10259,Julio,Willard,Greensboro,North Carolina,3,2023-01-11
2,A10260,Roslyn,Plott,Colombus,Ohio,3,2023-01-11
3,A10261,Eve,Haak,Detroit,Michigan,1,2023-01-11
4,A10262,Max,Kindle,Colombus,Ohio,2,2023-01-11


In [160]:
# DataSource1 preprocessing
merged_df = pd.merge(orderSource1, productSource1, on='OrderID', how='inner')

# Change abbreviation to full state name
cusState = merged_df['CustomerState']
cusState = cusState.replace(stateLookup.set_index('Abbreviation')['State'])
merged_df['CustomerState'] = cusState

# Split and Rename CustomerName into CustomerFirstName and CustomerLastName
cusName = merged_df['CustomerName'].str.split(' ', n=1, expand=True)
merged_df['CustomerFirstName'] = cusName[0]
merged_df['CustomerLastName'] = cusName[1]
merged_df.drop('CustomerName', axis=1, inplace=True)

# Reorder columns by alphabetical order
merged_df = merged_df.reindex(sorted(merged_df.columns), axis=1)

Data1 = merged_df
Data1.head()

,CustomerCity,CustomerFirstName,CustomerLastName,CustomerState,CustomerStatus,Discount,ExtendedPrice,FullPrice,OrderDate,OrderID,Product,ProductID,Quantity,TotalDiscount,UnitPrice
0,Pittsburgh,Suzan,Plock,Pennsylvania,Silver,0.0,168.0,168.0,2023-01-11,10248,Queso Cabrales,11,12,0.0,14.0
1,Pittsburgh,Suzan,Plock,Pennsylvania,Silver,0.0,98.0,98.0,2023-01-11,10248,Singaporean Hokkien Fried Mee,42,10,0.0,9.8
2,Pittsburgh,Suzan,Plock,Pennsylvania,Silver,0.0,174.0,174.0,2023-01-11,10248,Mozzarella di Giovanni,72,5,0.0,34.8
3,Miami,Allan,Strate,Florida,Platinum,0.0,1696.0,1696.0,2023-01-11,10249,Manjimup Dried Apples,51,40,0.0,42.4
4,Miami,Allan,Strate,Florida,Platinum,0.0,167.4,167.4,2023-01-11,10249,Tofu,14,9,0.0,18.6


In [161]:
# DataSource2 preprocessing
merged_df = pd.merge(orderSource2, productSource2, on='OrderID', how='inner')

# Replace 'A' in OrderID
orderID = merged_df['OrderID']
orderID = orderID.str.replace('A', '')

# Map CustomerStatus from 1,2,3 into Silver, Gold, Platinum
cusStatus = merged_df['CustomerStatus']
cusStatus = cusStatus.replace({1: 'Silver', 2: 'Gold', 3: 'Platinum'})
merged_df['CustomerStatus'] = cusStatus

# Calculate TotalDiscount by FullPrice * Discount
totalDiscount = merged_df['FullPrice'] * merged_df['Discount']
merged_df['TotalDiscount'] = totalDiscount

# Reorder columns by alphabetical order
merged_df = merged_df.reindex(sorted(merged_df.columns), axis=1)

Data2 = merged_df
Data2.head()


,CustomerCity,CustomerFirstName,CustomerLastName,CustomerState,CustomerStatus,Discount,ExtendedPrice,FullPrice,OrderDate,OrderID,Product,ProductID,Quantity,TotalDiscount,UnitPrice
0,Kansas City,Milagros,Fehrenbach,Kansas,Gold,0.2,608.00,760.0,2023-01-11,A10258,Chang,2,50,152.000002,15.2
1,Kansas City,Milagros,Fehrenbach,Kansas,Gold,0.2,122.88,153.6,2023-01-11,A10258,Mascarpone Fabioli,32,6,30.720000,25.6
2,Kansas City,Milagros,Fehrenbach,Kansas,Gold,0.2,884.00,1105.0,2023-01-11,A10258,Chef Anton's Gumbo Mix,5,65,221.000003,17.0
3,Greensboro,Julio,Willard,North Carolina,Platinum,0.0,80.00,80.0,2023-01-11,A10259,Sir Rodney's Scones,21,10,0.000000,8.0
4,Greensboro,Julio,Willard,North Carolina,Platinum,0.0,20.80,20.8,2023-01-11,A10259,Gravad lax,37,1,0.000000,20.8


In [162]:
# Concat or Append Data1 and Data2
AppendedData = pd.concat([Data1, Data2], ignore_index=True)
AppendedData.head()

# Create 'CustomerName' column by concatenating 'CustomerFirstName' and 'CustomerLastName'
customerName = AppendedData['CustomerFirstName'] + ' ' + AppendedData['CustomerLastName']
AppendedData['CustomerName'] = customerName
AppendedData.drop(['CustomerFirstName', 'CustomerLastName'], axis=1, inplace=True)

In [163]:
# Export as a new Excel file
AppendedData.to_excel('Lab1-Result-notebook.xlsx', index=False)